In [166]:
from musical_festival_lineup.musical_festival_lineup import MusicalFestivalSolution
from musical_festival_lineup.musical_festival_data import MusicalFestivalData
from musical_festival_lineup.utils import visualize_musical_lineup
import numpy as np
import time

In [167]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 1. Structuring the problem

The  objective  is  to  design  the  optimal  festival  lineup  by  scheduling  artists  across  stages  and 
time  slots while:
- Maximizing  prime  slot  popularity
- Ensuring  genre  diversity  among  stages 
- Minimizing  fan conflicts  at each time slot. 

__`Details`__

- **Artists**: An  artist is  characterized  by  a  popularity  score  (from  0  to  100) a genre  and  fan  base.
    <br>This information with be on __`artists`__ from musical_festival_data

- **Conflicts**:   with  other  artists  (score  from  0  to  1).  Conflicts  arise  when  artists  with  overlapping 
fan  bases  perform  at  the  same  time  but  on  different  stages,  which  can  negatively  impact 
attendance.
    <br>This information will be on __`conflits`__ from musical_festival_data


### 2. Defining a  solution/individual

Each  individual  represents  a  festival  lineup,  specifying  which  artist  plays  on  which  stage  and 
at  which  time  slot. 
These  are  the  constraints  : 
- Each artist is assigned to exactly one stage and slot 
- All time slots start and end at the same time. 
- All stages have the same amount of slots
- These artists should be distributed across  **5 stages  with  7 time slots each**

__`Important`__ 
<br>Any  lineup  where  an  artist  is  assigned  to  **multiple  time  slots  or 
left  unassigned** is  not  part  of  the  search  space  and  is  not  considered  a  solution.  It  is 
forbidden to generate such an arrangement during evolution. 


__`we propose a  having  a solution as`__:

- A string of 5 (stages)*7(slots)= 35 numbers of **1** to **35**
- As for the genetic algorithms, we should have string of the same size, we will be appending a 0 to 1 digit numbers.
- So the solution will be a string of 35 * 2 (two character)= so will have i string of 70

Here a individual of our problem:

In [168]:
data=MusicalFestivalData()


In [169]:
# Python code to show a random initialized solution
lineup=MusicalFestivalSolution(data=data)
lineup.repr

'0714342204191812012916000821103115030902231117252005242632301306332827'

- Having that, we have to options:
- 1. The first 7 numbers represent the stage 1 , having each of the 7 representing the slots in  order (1 to 7) by the slot?
- 2. Or the first 5 would represent the slot 1 (first time slot) , having the each of the 5 representing the stages in order (1 to 5)?

For that we choose the option 2, because how our fitness will be defined. It is well explained in the session 3, when defining fitness


In [170]:
visualize_musical_lineup(lineup.repr,data.artists)

,Slot,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5
0,Slot 1,7: Static Mirage|Rock|94,14: Synthwave Saints|Rock|94,34: Parallel Dimension|Electronic|58,22: Quantum Beat|Hip-Hop|96,4: The Silver Owls|Classical|85
1,Slot 2,19: Astral Tide|Electronic|69,18: Velvet Underground|Rock|72,12: Blue Horizon|Pop|51,1: Solar Flare|Electronic|78,29: Harmonic Dissonance|Classical|96
2,Slot 3,16: Mystic Rhythms|Classical|78,0: Midnight Echo|Rock|75,8: Crimson Harmony|Classical|20,21: Celestial Voyage|Electronic|95,10: The Wandering Notes|Jazz|84
3,Slot 4,31: The Jazz Nomads|Jazz|64,15: Golden Ember|Rock|61,3: Neon Reverie|Electronic|100,9: Deep Resonance|Jazz|90,2: Velvet Pulse|Jazz|35
4,Slot 5,23: Electric Serpents|Electronic|99,11: Phantom Groove|Hip-Hop|47,17: Nightfall Sonata|Classical|84,25: Rhythm Alchemy|Jazz|94,20: The Sonic Drifters|Rock|88
5,Slot 6,5: Echo Chamber|Electronic|98,24: Shadow Cadence|Jazz|66,26: Cloud Nine Collective|Pop|97,32: The Bassline Architects|Hip-Hop|61,30: Turbo Vortex|Rock|53
6,Slot 7,13: Lunar Spectrum|Rock|99,6: Aurora Skies|Pop|75,33: Cosmic Frequency|Rock|53,28: The Polyrhythm Syndicate|Jazz|66,27: Hypnotic Echoes|Rock|77


### 3. Defining fitness

" The  quality  of  a  festival  lineup  is  determined  by  balancing  __`three  equally`__  important 
objectives** ,  each  contributing  to  an  overall  score.  Since  these  objectives  have  different 
scales,  ** they  must  be  normalized  to  the  same  range  (between  0  and  1)  to  ensure  they 
contribute equally to the final fitness score.
<br>
 These are the objectives: 
 
__`Prime  Slot  Popularity`__  :  The  most  popular  artists  should  be  scheduled  in  the  prime 
slots  (**the  last  time  slot  on  each  stage**).  This  score  is  calculated  by  normalizing  the 
total  popularity  of  artists  in  prime  slots  against  the  maximum  possible  total  popularity 
(e.g. if only most popular artists - score 100 - were scheduled on the prime slot). 

__`Genre  Diversity`__  :  A  diverse  distribution  of  **genres  among  stages  in  each  time  slot**
improves  the  festival  experience.  This  score  is  determined  by  normalizing  the 
number  of  unique  genres  in  each  slot  relative  to  the  maximum  possible  number  of 
unique  genres  (e.g.  if  only  different  genres  were  scheduled  in  that  time  slot)  .  Then 
you take the average among time slots. 

__`Conflict  Penalty`__  : ** Fan  conflicts  occur  when  artists  with  overlapping  audiences 
perform  at  the  same  time  on  different  stages**. 
 This  score  is  calculated  by  normalizing 
the  total  conflict  value  in  each  slot  against  the  worst  possible  conflict  scenario  (e.g. 
where  all  artists  with  maximum  conflict  are  scheduled  together).  Then  you  take  the 
average  among  time  slots.  Since  conflicts  negatively  impact  the  lineup,  this  is  a 
penalization  score. 
"

__`Prime  Slot  Popularity`__  

1.  Maximum  possible  total  popularity, having into account that There is 5 stages, so we eill be summing the 5(NUM STAGES) largest

In [171]:
data.max_popularity_in_prime_slot

493

2. Total  popularity  of  artists: <br>
For that, it was defined a function that receives artists Ids and returns the popularity for that specifics Ids. <br>
Lets test it!



In [172]:
#Summing popularity of all artists
data.get_sum_popularity(list(range(14)))

1031

In [173]:
#Summing popularity of [1,2,3,4]
data.get_sum_popularity([1,2,3,4])

298

3. Normalizing the popularity against the max number of popularity

In [174]:
lineup._get_popularity_normalized([1,2,3,4])

0.6044624746450304

__`Genre  Diversity`__  : 

1. Get maximum number of genres per slot, can not be greater than the total number of stages, which is 5

In [175]:
#Genre  Diversity
data.max_distinct_genre_per_slot

5

2. Number of distinct genres : <br>
For that, it was defined a function that receives artists Ids and returns distinct genres in that list of specifics Ids. <br>
Lets test it!


In [176]:
data.get_count_distinct_genres(artists_ids_list=[0,1,10,2])

3

3. Normalizing the genres against the max number of distinct genres

In [177]:
lineup._get_genre_diversity_normalized(artists_ids_list=[0,1,10])

0.6

__`Penalty Conflict`__  : 

1. Get maximum possible conflift, the  worst  possible  conflict  scenario, so for that we get the top K worst case scenerio. <br>
 What is K?
 Is the numbers os conflits in a slot, which is the combination of 5 elements when grouping in two so, we will have C5,2, tthat is equal to 10.
 So will will be choosing the top 10 worst conflits


In [178]:
data.max_worst_conflit_per_slot

np.float64(10.0)

2. The  Total  conflict  value  in  each  slot : <br>
For that, it was defined a function that receives artists Ids and sum of all conflicts in the list. <br>
Lets test it!


In [179]:
data.get_sum_conflicts(artists_ids_list=[3,1,10])

np.float64(1.8)

In [180]:
data.get_sum_conflicts(artists_ids_list=[32, 29, 25, 13, 30])

np.float64(5.75)

In [181]:
data.get_sum_conflicts(artists_ids_list=[2,10])

np.float64(0.9)

3. Normalizing the conflicts against the worst conflit scenario

In [182]:
lineup._get_conflicts_normalized(artists_ids_list=[3,1,10])

np.float64(0.18)

In [183]:
lineup._get_conflicts_normalized(artists_ids_list=[32, 29, 25, 13, 30])

np.float64(0.575)

##### 3.2 Other important functions in the solution

In [184]:
lineup.repr

'0714342204191812012916000821103115030902231117252005242632301306332827'

- Get the list of artists Ids in a Slot, Slot is a parameter

In [185]:
lineup._get_slot_repr_list(1)

[19, 18, 12, 1, 29]

In [186]:
lineup._get_slot_repr_list(0)

[7, 14, 34, 22, 4]

##### 3.3 Fitness

In [187]:
#Random 1
lineup=MusicalFestivalSolution(data=data)
lineup.repr
lineup.fitness(verbose=True)

Slot of artists: [3, 8, 27, 12, 32]
Slot 0: Conflitcs: 0.5, genres: 1.0, sum_popularity: 0
Slot 0: List of  Conflitcs: [np.float64(0.5)], List of genres: [1.0], Popularity of the prime slot: 0
Slot of artists: [11, 23, 30, 34, 6]
Slot 1: Conflitcs: 0.635, genres: 0.8, sum_popularity: 0
Slot 1: List of  Conflitcs: [np.float64(0.5), np.float64(0.635)], List of genres: [1.0, 0.8], Popularity of the prime slot: 0
Slot of artists: [20, 21, 22, 9, 7]
Slot 2: Conflitcs: 0.5200000000000001, genres: 0.8, sum_popularity: 0
Slot 2: List of  Conflitcs: [np.float64(0.5), np.float64(0.635), np.float64(0.5200000000000001)], List of genres: [1.0, 0.8, 0.8], Popularity of the prime slot: 0
Slot of artists: [28, 26, 1, 2, 18]
Slot 3: Conflitcs: 0.6300000000000001, genres: 0.8, sum_popularity: 0
Slot 3: List of  Conflitcs: [np.float64(0.5), np.float64(0.635), np.float64(0.5200000000000001), np.float64(0.6300000000000001)], List of genres: [1.0, 0.8, 0.8, 0.8], Popularity of the prime slot: 0
Slot of arti

np.float64(0.8841104027818023)

In [188]:
#Random 2
lineup=MusicalFestivalSolution(data=data)
lineup.repr
lineup.fitness(verbose=True)

Slot of artists: [20, 25, 9, 12, 34]
Slot 0: Conflitcs: 0.4, genres: 0.8, sum_popularity: 0
Slot 0: List of  Conflitcs: [np.float64(0.4)], List of genres: [0.8], Popularity of the prime slot: 0
Slot of artists: [1, 11, 30, 8, 2]
Slot 1: Conflitcs: 0.225, genres: 1.0, sum_popularity: 0
Slot 1: List of  Conflitcs: [np.float64(0.4), np.float64(0.225)], List of genres: [0.8, 1.0], Popularity of the prime slot: 0
Slot of artists: [29, 24, 10, 15, 6]
Slot 2: Conflitcs: 0.4800000000000001, genres: 0.8, sum_popularity: 0
Slot 2: List of  Conflitcs: [np.float64(0.4), np.float64(0.225), np.float64(0.4800000000000001)], List of genres: [0.8, 1.0, 0.8], Popularity of the prime slot: 0
Slot of artists: [28, 21, 13, 27, 0]
Slot 3: Conflitcs: 0.43, genres: 0.6, sum_popularity: 0
Slot 3: List of  Conflitcs: [np.float64(0.4), np.float64(0.225), np.float64(0.4800000000000001), np.float64(0.43)], List of genres: [0.8, 1.0, 0.8, 0.6], Popularity of the prime slot: 0
Slot of artists: [7, 16, 26, 4, 31]
Slo

np.float64(0.9650536076499565)

In [189]:
#Random 3
lineup=MusicalFestivalSolution(data=data)
lineup.repr
lineup.fitness(verbose=True)

Slot of artists: [25, 18, 9, 32, 19]
Slot 0: Conflitcs: 0.3, genres: 0.8, sum_popularity: 0
Slot 0: List of  Conflitcs: [np.float64(0.3)], List of genres: [0.8], Popularity of the prime slot: 0
Slot of artists: [4, 7, 3, 2, 28]
Slot 1: Conflitcs: 0.675, genres: 0.8, sum_popularity: 0
Slot 1: List of  Conflitcs: [np.float64(0.3), np.float64(0.675)], List of genres: [0.8, 0.8], Popularity of the prime slot: 0
Slot of artists: [1, 22, 13, 5, 24]
Slot 2: Conflitcs: 0.29000000000000004, genres: 0.8, sum_popularity: 0
Slot 2: List of  Conflitcs: [np.float64(0.3), np.float64(0.675), np.float64(0.29000000000000004)], List of genres: [0.8, 0.8, 0.8], Popularity of the prime slot: 0
Slot of artists: [15, 30, 11, 17, 12]
Slot 3: Conflitcs: 0.32499999999999996, genres: 0.8, sum_popularity: 0
Slot 3: List of  Conflitcs: [np.float64(0.3), np.float64(0.675), np.float64(0.29000000000000004), np.float64(0.32499999999999996)], List of genres: [0.8, 0.8, 0.8, 0.8], Popularity of the prime slot: 0
Slot of

np.float64(1.1895552013909012)